In [1]:
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Point, Polygon

import re,os
import numpy as np

import kml2geojson
import json
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

/Users/ahakso/.virtualenvs/detroit/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


### Convert KML to geojson

In [2]:
kml2geojson.convert('Current_City_of_Detroit_Neighborhoods.kml', 'Current_City_of_Detroit_Neighborhoods.json')
kml2geojson.convert('DDOT_Bus_Stops.kml', 'DDOT_Bus_Stops.json')

### Read KML

In [17]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file('Current_City_of_Detroit_Neighborhoods.kml', driver='KML')

In [4]:
#fiona.drvsupport.supported_drivers['KML'] = 'rw'
#fiona.drvsupport.supported_drivers['kml'] = 'rw' 
#neighborhood = gpd.read_file('Current_City_of_Detroit_Neighborhoods.kml', driver='KML')
#buses = gpd.read_file('DDOT_Bus_Stops.kml', driver='KML')
neighborhood = gpd.read_file('Current_City_of_Detroit_Neighborhoods.json/Current_City_of_Detroit_Neighborhoods.geojson')
buses = gpd.read_file('DDOT_Bus_Stops.json/DDOT_Bus_Stops.geojson')

### Convert to dataframe

In [5]:
neighborhood_df = pd.DataFrame(neighborhood)
buses_df = pd.DataFrame(buses)

### sjoin exploration

In [6]:
neighborhood.head()

,fill-opacity,stroke,stroke-opacity,OBJECTID,council_district,nhood_num,nhood_name,Website,geometry
0,0.0,#ff0000,1.0,1,4,103,West End,https://theneighborhoods.org/neighborhoods/wes...,"POLYGON ((-82.99152 42.38439, -82.99249 42.384..."
1,0.0,#ff0000,1.0,2,4,105,Chandler Park,https://theneighborhoods.org/neighborhoods/cha...,"POLYGON ((-82.97585 42.38933, -82.97603 42.389..."
2,0.0,#ff0000,1.0,3,4,100,Riverbend,https://theneighborhoods.org/neighborhoods/riv...,"POLYGON ((-82.96784 42.38369, -82.96783 42.383..."
3,0.0,#ff0000,1.0,4,4,101,Jefferson Chalmers,https://theneighborhoods.org/neighborhoods/jef...,"POLYGON ((-82.95141 42.35503, -82.95292 42.354..."
4,0.0,#ff0000,1.0,5,1,29,Riverdale,https://theneighborhoods.org/neighborhoods/riv...,"POLYGON ((-83.25748 42.41460, -83.25729 42.411..."


In [7]:
buses.head()

,FID,location,position,StreetDire,Latitude,Longitude,OnStreet,AtStreet,LocationOr,Shelter,Bench,Trash,Date,StopID,geometry
0,1,Fenkell & Snowden,Farside,EB,42.402042,-83.176779,Fenkell,Snowden,,0,0,0,2019-11-16T00:00:00+00:00,5172,POINT (-83.17678 42.40204)
1,2,Fenkell & Schaefer,Farside,EB,42.402004,-83.178746,Fenkell,Schaefer,,0,0,0,2019-11-16T00:00:00+00:00,260,POINT (-83.17875 42.40200)
2,3,Fenkell & Lesure,Farside,EB,42.40198,-83.180987,Fenkell,Lesure,,0,0,0,2019-11-16T00:00:00+00:00,5166,POINT (-83.18099 42.40198)
3,4,Fenkell & Cruse,Farside,EB,42.401917,-83.183147,Fenkell,Cruse,,0,0,0,2019-11-16T00:00:00+00:00,5164,POINT (-83.18315 42.40192)
4,5,Fenkell & Freeland,Farside,EB,42.401852,-83.185212,Fenkell,Freeland,,0,0,0,2019-11-16T00:00:00+00:00,5162,POINT (-83.18521 42.40185)


In [8]:
bus_stops_with_neighborhood = gpd.sjoin(buses, neighborhood, how='left', op='intersects')

In [9]:
bus_stops_with_neighborhood

,FID,location,position,StreetDire,Latitude,Longitude,OnStreet,AtStreet,LocationOr,Shelter,Bench,Trash,Date,StopID,geometry,index_right,fill-opacity,stroke,stroke-opacity,OBJECTID,council_district,nhood_num,nhood_name,Website
0,1,Fenkell & Snowden,Farside,EB,42.402042,-83.176779,Fenkell,Snowden,,0,0,0,2019-11-16T00:00:00+00:00,5172,POINT (-83.17678 42.40204),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
1,2,Fenkell & Schaefer,Farside,EB,42.402004,-83.178746,Fenkell,Schaefer,,0,0,0,2019-11-16T00:00:00+00:00,260,POINT (-83.17875 42.40200),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
2,3,Fenkell & Lesure,Farside,EB,42.40198,-83.180987,Fenkell,Lesure,,0,0,0,2019-11-16T00:00:00+00:00,5166,POINT (-83.18099 42.40198),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
3,4,Fenkell & Cruse,Farside,EB,42.401917,-83.183147,Fenkell,Cruse,,0,0,0,2019-11-16T00:00:00+00:00,5164,POINT (-83.18315 42.40192),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
4,5,Fenkell & Freeland,Farside,EB,42.401852,-83.185212,Fenkell,Freeland,,0,0,0,2019-11-16T00:00:00+00:00,5162,POINT (-83.18521 42.40185),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,5542,St Cyril & Georgia,Farside,NB,42.3938554,-83.0271958,St Cyril,Georgia,,0,0,0,2019-11-16T00:00:00+00:00,60157,POINT (-83.02720 42.39386),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...
5542,5543,St Cyril & Marcus,Farside,SB,42.3957358,-83.0269471,St Cyril,Marcus,,0,0,0,2019-11-16T00:00:00+00:00,60158,POINT (-83.02695 42.39574),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...
5543,5544,St Cyril & Marcus,Nearside,NB,42.395889,-83.0267607,St Cyril,Marcus,,0,0,0,2019-11-16T00:00:00+00:00,60159,POINT (-83.02676 42.39589),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...
5544,5545,St Cyril & Majorie,Farside,SB,42.3992497,-83.0256015,St Cyril,Majorie,,0,0,0,2019-11-16T00:00:00+00:00,60160,POINT (-83.02560 42.39925),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...


In [10]:
bus_stops_with_neighborhood.shape

(5546, 24)

In [11]:
bus_stops_with_neighborhood['OBJECTID'].isna().sum()

595

# Read all

In [12]:
fns = [x.replace('.csv','') for x in os.listdir() if 'csv' in x]

data = {}
for f in fns:
    data[f] = pd.read_csv(f"{f}.csv")
    data[f] = data[f].loc[:,data[f].notnull().sum()!=0]

/Users/ahakso/.virtualenvs/detroit/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (9,11,15,16,17,30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/ahakso/.virtualenvs/detroit/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
data['Blight_Violations'].head()

,X,Y,ticket_id,ticket_number,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,violator_id,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,country,violation_date,ticket_issued_time,hearing_date,hearing_time,judgment_date,violation_code,violation_description,disposition,payment_date,payment_status,collection_status,violation_address,parcelno,oid
0,-83.072573,42.383350,18645,05001700DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Dean Byrd,601,KING,NaN,1614,599,King,Det,MI,48202,NaN,NaN,2005/01/13 00:00:00+00,1900/01/01 14:00:00,2005/02/07 00:00:00+00,09:00:00,2005/02/07 00:00:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,1970/01/01 00:00:00+00,NaN,NaN,601 KING,03002390.,18645
1,-83.072479,42.383394,18646,05001701DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Cynthia Roberts,607,KING,NaN,1615,18604,Appoline,Det,MI,48235,NaN,NaN,2005/01/14 00:00:00+00,1900/01/01 00:45:00,2005/02/14 00:00:00+00,09:00:00,2019/09/16 10:26:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,1970/01/01 00:00:00+00,NO PAYMENT DUE,NaN,607 KING,03002391.,18646
2,-83.115061,42.359916,18648,05001703DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Dannny Barnes,4066,COLUMBUS,NaN,1617,4066,Columbus,Det,MI,48204,NaN,NaN,2005/01/14 00:00:00+00,1900/01/01 14:00:00,2005/02/14 00:00:00+00,09:00:00,2005/02/14 00:00:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,1970/01/01 00:00:00+00,NO PAYMENT DUE,NaN,4066 COLUMBUS,14002572.,18648
3,-83.128041,42.393448,18649,05001704DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Gloria Seldon,3005,PASADENA,NaN,1618,3005,Pasada,Det,MI,48238,NaN,NaN,2005/01/14 00:00:00+00,1900/01/01 14:50:00,2005/02/14 00:00:00+00,09:00:00,2019/09/16 10:26:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,1970/01/01 00:00:00+00,NO PAYMENT DUE,NaN,3005 PASADENA,12004708.,18649
4,NaN,NaN,18650,05001705DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Bessie Thomas,4024,CLEMENTS,NaN,1619,20211,Westmoreland,Det,MI,48219,NaN,NaN,2005/01/14 00:00:00+00,1900/01/01 14:05:00,2005/02/14 00:00:00+00,09:00:00,2019/09/16 10:26:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,1970/01/01 00:00:00+00,NO PAYMENT DUE,NaN,4024 CLEMENTS,NaN,18650


In [19]:
df = gpd.GeoDataFrame(
    data['Blight_Violations'], geometry=gpd.points_from_xy(data['Blight_Violations']['X'], data['Blight_Violations']['Y']))

In [16]:
data.keys()

dict_keys(['DDOT_Bus_Stops', 'Blight_Violations', 'Current_City_of_Detroit_Neighborhoods', 'Rental_Statuses', 'Certificates_Of_Compliance', 'SMART_Bus_Stops', 'DPD_Citizen_Complaints', '911_Calls_For_Service'])

In [ ]:
display(neighborhood.head(2))
df.head(2)

In [ ]:
t = gpd.sjoin(neighborhood.loc[:,['nhood_name','geometry']], df, how='left', op='intersects')

In [ ]:
df.Y.notna().sum()

In [ ]:
t.nhood_name.value_counts()